In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup 
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [3]:
train_df = pd.read_csv('labeledTrainData.tsv','\t',quoting=3)
test_df = pd.read_csv('testData.tsv','\t',quoting=3)

In [4]:
train_df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
train_df['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

 we are having only two classification

### Pre Processing Pipeline

In [6]:
#%%
from multiprocessing import Pool,cpu_count

def Preprocessing_Pipe(text):


        newtext =" ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())

    ## Making text to Lower case

        text = newtext.lower()

    ## Removing numbers

        newtext= re.sub(r'\d+',"",text)


    ## removing punctuation

        trans = str.maketrans("","",string.punctuation)
        text = newtext.translate(trans)
    #Tokenizing

        text = word_tokenize(text)


    # Removing Stop Words

        text  = [i for i in text if not i in stopwords.words('english')]


    #Lemmatize

        word_lem = WordNetLemmatizer()
    #we are using recursive call since for eg: calling can be writen as (called,caller,call). So we are lemmatizing into the word call

        text = [word_lem.lemmatize(token) for token in text]
        text = " ".join(text)

        return text




In [7]:
c =[]
m = 0
for i in train_df['review']:
    resul = Preprocessing_Pipe(i)
    if m%1000 == 0:
        print(m)
    m = m+1
    c.append(resul)

        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [8]:
"""def Preprocessing_Pipe(db):
    train_preprocess = []
    for i in db:
        text = remove_url(text)
        text = lower_case(text)
        text = remove_num(text)
        text = remove_punc(text)
        text = tokenize(text)
        text = Lemmatize(text)
        text = remove_stop(text)
        text = " ".join(text)
        time.sleep(1)
        train_preprocess.append(text)
        return text"""

'def Preprocessing_Pipe(db):\n    train_preprocess = []\n    for i in db:\n        text = remove_url(text)\n        text = lower_case(text)\n        text = remove_num(text)\n        text = remove_punc(text)\n        text = tokenize(text)\n        text = Lemmatize(text)\n        text = remove_stop(text)\n        text = " ".join(text)\n        time.sleep(1)\n        train_preprocess.append(text)\n        return text'

In [9]:
train_df['processed_text'] = c

In [10]:
train_df.head()

,id,sentiment,review,processed_text
0,"""5814_8""",1,"""With all this stuff going down at the moment ...",stuff going moment mj started listening music ...
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ...",classic war world timothy hines entertaining f...
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell...",film start manager nicholas bell giving welcom...
3,"""3630_4""",0,"""It must be assumed that those who praised thi...",must assumed praised film greatest filmed oper...
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ...",superbly trashy wondrously unpretentious explo...


In [11]:
test =[]
m = 0
for j in test_df['review']:
    resul = Preprocessing_Pipe(j)
    if m%5000 == 0:
        print(m)
    m = m+1
    test.append(resul)

0
5000
10000
15000
20000


In [12]:
test_df['processed_text'] = test

In [13]:
test_df.head()


,id,review,processed_text
0,"""12311_10""","""Naturally in a film who's main themes are of ...",naturally film main theme mortality nostalgia ...
1,"""8348_2""","""This movie is a disaster within a disaster fi...",movie disaster within disaster film full great...
2,"""5828_4""","""All in all, this is a movie for kids. We saw ...",movie kid saw tonight child loved one point ki...
3,"""7186_2""","""Afraid of the Dark left me with the impressio...",afraid dark left impression several different ...
4,"""12128_7""","""A very accurate depiction of small time mob l...",accurate depiction small time mob life filmed ...


In [14]:
trainlist = list(train_df['processed_text'])
testlist = list(test_df['processed_text'])
corpus = trainlist+testlist

In [15]:
corpus

['stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighty maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle message mj feeling towards press also obvious message drug bad kay br br visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fan would say made fan true really nice br br actual feature film bit finally start minute excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord want mj dead bad beyond mj overheard plan nah joe pesci character ranted wanted people know supplying drug etc dunno maybe hate mj music br br lot cool thing like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually director hate wo

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
tfidf = TfidfVectorizer()

In [18]:
corpus_fit = tfidf.fit(corpus)

In [63]:
corpus_fit.transform(train_df['processed_text']).shape

(25000, 89667)

In [19]:
train_transform = tfidf.transform(train_df['processed_text'])
test_transform = tfidf.transform(test_df['processed_text'])

In [51]:
train_transform

(25000, 89667)

In [20]:
y_actual = train_df['sentiment']

In [21]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression()

In [22]:
reg.fit(train_transform,y_actual)

LogisticRegression()

In [23]:
Y_pred = reg.predict(test_transform)

In [24]:
Y_pred

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [25]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_actual,Y_pred))

0.5016


In [26]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(reg,train_transform,y_actual,cv=3)

In [27]:
score

array([0.88960883, 0.88059522, 0.88347534])

In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_actual,Y_pred))

              precision    recall  f1-score   support

           0       0.50      0.50      0.50     12500
           1       0.50      0.50      0.50     12500

    accuracy                           0.50     25000
   macro avg       0.50      0.50      0.50     25000
weighted avg       0.50      0.50      0.50     25000



In [29]:
from sklearn.svm import SVC

SVM_m = SVC(kernel = 'linear')

SVM_m.fit(train_transform,y_actual)

SVC(kernel='linear')

In [30]:
pred = SVM_m.predict(test_transform) 

In [31]:
pred

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [32]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(SVM_m,train_transform,y_actual,cv=3)

In [33]:
score

array([0.89608831, 0.88455538, 0.88803552])

In [34]:
from sklearn.ensemble import RandomForestClassifier


clss = RandomForestClassifier(n_estimators=100)
clss.fit(train_transform,y_actual)


RandomForestClassifier()

In [35]:
c_pred = clss.predict(test_transform) 

In [36]:
print(accuracy_score(y_actual,c_pred))

0.50116


In [37]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(clss,train_transform,y_actual,cv=3)

In [38]:
score

array([0.84365251, 0.83763351, 0.84987399])

im taking SVM model for deployement

In [39]:
result = pd.read_csv('sampleSubmission.csv')
result['sentiment'] = pred
result.to_csv('sampleSubmission.csv',index = False)

In [40]:
import pickle
pickle.dump(reg,open('nlp_model.pkl','wb'))

In [55]:
#going to take train trasform import as pickle

pickle.dump(train_transform,open('transform.pkl','wb'))

In [56]:
pickle.dump(corpus_fit,open('corpus.pkl','wb'))

In [59]:
m =pickle.load(open('corpus.pkl','rb'))

{'stuff': 76027,
 'going': 32419,
 'moment': 51798,
 'mj': 51511,
 'started': 75008,
 'listening': 46068,
 'music': 53039,
 'watching': 86389,
 'odd': 55701,
 'documentary': 22075,
 'watched': 86384,
 'wiz': 87808,
 'moonwalker': 52093,
 'maybe': 49210,
 'want': 86213,
 'get': 31569,
 'certain': 12651,
 'insight': 39427,
 'guy': 33961,
 'thought': 79451,
 'really': 64202,
 'cool': 16488,
 'eighty': 24173,
 'make': 47861,
 'mind': 50961,
 'whether': 87022,
 'guilty': 33747,
 'innocent': 39331,
 'part': 58005,
 'biography': 7824,
 'feature': 27669,
 'film': 28222,
 'remember': 65257,
 'see': 69902,
 'cinema': 14012,
 'originally': 56497,
 'released': 65135,
 'subtle': 76327,
 'message': 50346,
 'feeling': 27726,
 'towards': 80611,
 'press': 61588,
 'also': 2126,
 'obvious': 55597,
 'drug': 23037,
 'bad': 5428,
 'kay': 42416,
 'br': 9426,
 'visually': 85577,
 'impressive': 38437,
 'course': 17047,
 'michael': 50550,
 'jackson': 40615,
 'unless': 83393,
 'remotely': 65314,
 'like': 45785,


In [74]:
import pickle
print(pickle.__version__)

AttributeError: module 'pickle' has no attribute '__version__'